# 2.2 Data Preprocessing

Have been working with synthetic data that arrived in ready-made tensors. 

However, to apply deep learning in the wild we must extract messy data stored in arbitrary formats, and preprocess it to suit our needs. Fortunately, the **pandas** library can do much of the heavy lifting. 

This section, while no substitute for a proper pandas tutorial, will give you a crash course on some of the most common routines.

### 2.2.1 Reading the Dataset

- Comma-separated values (CSV) files are ubiquitous for the storing of tabular (spreadsheet-like) data. 

- Each line corresponds to one record and consists of several (comma-separated) fields 

- e.g., “Albert Einstein,March 14 1879,Ulm,Federal polytechnic school,field of gravitational physics”. 

- To demonstrate how to load CSV files with pandas, we create a CSV file below . ./data/house_tiny.csv. 

- This file represents a dataset of homes, where each row corresponds to a distinct home and the columns correspond to the number of rooms (NumRooms), the roof type (RoofType), and the price (Price).

In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('''
    NumRooms,RoofType,Price
    NA,NA,127500
    2,NA,106000
    4,Slate,178100
    NA,NA,140000''')

This code snippet creates a CSV file named house_tiny.csv in a directory named 'data' (which is created if it doesn't already exist) in the parent directory of the current working directory.

- **import os**: Imports the Python os module, which provides a way to interact with the operating system.

- **os.makedirs(os.path.join('..', 'data'), exist_ok=True)**: Creates a directory named data in the parent directory of the current working directory. The os.makedirs() function is used for creating directories recursively. The exist_ok=True argument ensures that the function doesn't raise an error if the directory already exists.

- **data_file = os.path.join('..', 'data', 'house_tiny.csv')**: Constructs the file path for the CSV file. It concatenates the directory path ../data with the filename house_tiny.csv using os.path.join().

- **with open(data_file, 'w') as f:**: Opens the CSV file house_tiny.csv in write mode. The with statement ensures that the file is properly closed after writing. It assigns the file object to the variable f.

- **f.write(''' ... ''')**: Writes the content enclosed in triple single quotes to the CSV file. The content consists of four lines, each representing a row in the CSV file. Each row contains values separated by commas: NumRooms, RoofType, and Price. The last character before the closing triple quotes is a newline character, ensuring that each row is written on a new line in the file.

The first row contains column headers.
The subsequent rows represent data entries for the respective columns.


In summary, this code snippet creates a CSV file named house_tiny.csv with sample data representing the number of rooms (NumRooms), roof type (RoofType), and price (Price) of houses. The file is created in a directory named data in the parent directory of the current working directory.

import pandas and load the dataset with read_csv

In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

      NumRooms RoofType   Price
0           NA      NaN  127500
1            2      NaN  106000
2            4    Slate  178100
3           NA      NaN  140000


### 2.2 Data Preperation

In supervised learning, we train models to predict a designated *target* value, given some set of *input* values. 

First step in processing the dataset is to separate out columns corresponding to input versus target values. 
We can select columns either by name or via **integer-location based indexing (`iloc`)**.

You might have noticed that `pandas` replaced all CSV entries with value `NA` with a special `NaN` (*not a number*) value. 

This can also happen whenever an entry is empty,e.g., "3,,,270000".
These are called *missing values* and they are a persistent menace that you will confront throughout your career. 

Depending upon the context, missing values might be handled either via **imputation** or **deletion**.

Imputation replaces missing values with estimates of their values while deletion simply discards either those rows or those columns that contain missing values. 

Here are some common imputation heuristics:

[**For categorical input fields, we can treat `NaN` as a category.**]

Since the `RoofType` column takes values `Slate` and `NaN`,
`pandas` can convert this column into two columns `RoofType_Slate` and `RoofType_nan`.

A row whose roof type is `Slate` will set values 
of `RoofType_Slate` and `RoofType_nan` to 1 and 0, respectively.

The converse holds for a row with a missing `RoofType` value.

In [4]:
inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

       NumRooms_    2      NumRooms_    4      NumRooms_    NA  \
0               False               False                 True   
1                True               False                False   
2               False                True                False   
3               False               False                 True   

       NumRooms_nan  RoofType_Slate  RoofType_nan  
0             False           False          True  
1             False           False          True  
2             False            True         False  
3             False           False          True  


- **inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]**: 
    - This line extracts the input features and the target variable from the DataFrame 'data'.
    - **data.iloc[:, 0:2]** selects all rows and the first two columns of the DataFrame, representing the input features.
    - **data.iloc[:, 2]** selects all rows of the DataFrame and the third column, representing the target variable.
    - The extracted input features are assigned to the variable inputs, and the target variable is assigned to the variable targets.

- **inputs = pd.get_dummies(inputs, dummy_na=True)**:
    - This line creates dummy variables for categorical features in the inputs DataFrame using one-hot encoding.
    - **pd.get_dummies()** is a Pandas function used for one-hot encoding categorical variables.
    - By specifying **dummy_na=True**, it also creates dummy variables for missing values (NaNs) in the input data.
    - The resulting DataFrame with dummy variables is assigned back to the variable inputs.

For missing numerical values, 
one common heuristic is to 
[**replace the `NaN` entries with 
the mean value of the corresponding column**].


In [9]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

       NumRooms_    2      NumRooms_    4      NumRooms_    NA  \
0               False               False                 True   
1                True               False                False   
2               False                True                False   
3               False               False                 True   

       NumRooms_nan  RoofType_Slate  RoofType_nan  
0             False           False          True  
1             False           False          True  
2             False            True         False  
3             False           False          True  


output should look like this after imputation
   - Index,      NumRooms,      RoofType_Slate,      RoofType_nan,
- 0,       3.0,           False,          True
- 1,       2.0,           False ,         True
- 2 ,      4.0   ,         True   ,      False
- 3 ,      3.0  ,         False ,         True

### 2.2.3 Conversion to the Tensor Format
Now that all the entries in inputs and targets are numerical, we can load them into a tensor

In [13]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(targets.to_numpy(dtype=float))
X, y

(tensor([[0., 0., 1., 0., 0., 1.],
         [1., 0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

In [14]:
type(X)

torch.Tensor

- **X = torch.tensor(inputs.to_numpy(dtype=float))**:
    - **inputs.to_numpy(dtype=float)** converts the DataFrame inputs to a NumPy array with data type float.
    - **torch.tensor()** converts the NumPy array to a PyTorch tensor X.

- **y = torch.tensor(targets.to_numpy(dtype=float))**:
    - **targets.to_numpy(dtype=float)** converts the DataFrame targets to a NumPy array with data type float.
    - **torch.tensor()** converts the NumPy array to a PyTorch tensor y.

In [15]:
type(inputs.to_numpy(dtype=float))

numpy.ndarray

### 2.2.4 Discussion

Now know how to partition data columns, impute missing variables, 
and load `pandas` data into tensors. 
 
While this crash course kept things simple, data processing can get hairy.

For example, rather than arriving in a single CSV file, our dataset might be spread across multiple files extracted from a relational database.

For instance, in an e-commerce application, customer addresses might live in one table and purchase data in another.

Moreover, practitioners face myriad data types beyond categorical and numeric, for example, text strings, images, audio data, and point clouds. 

Oftentimes, advanced tools and efficient algorithms are required in order to prevent data processing from becoming the biggest bottleneck in the machine learning pipeline. 

We must pay attention to data quality. Real-world datasets are often plagued by outliers, faulty measurements from sensors, and recording errors, which must be addressed before feeding the data into any model. 

Data visualization tools such as [seaborn](https://seaborn.pydata.org/), 
[Bokeh](https://docs.bokeh.org/), or [matplotlib](https://matplotlib.org/)
can help you to manually inspect the data and develop intuitions about 
the type of problems you may need to address.

### 2.2.3 Exercises

1. Try loading datasets, e.g., Abalone from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) and inspect their properties. What fraction of them has missing values? What fraction of the variables is numerical, categorical, or text?
1. Try indexing and selecting data columns by name rather than by column number. The pandas documentation on [indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) has further details on how to do this.
1. How large a dataset do you think you could load this way? What might be the limitations? Hint: consider the time to read the data, representation, processing, and memory footprint. Try this out on your laptop. What happens if you try it out on a server? 
1. How would you deal with data that has a very large number of categories? What if the category labels are all unique? Should you include the latter?
1. What alternatives to pandas can you think of? How about [loading NumPy tensors from a file](https://numpy.org/doc/stable/reference/generated/numpy.load.html)? Check out [Pillow](https://python-pillow.org/), the Python Imaging Library.
